In [49]:
from binance.client import Client
from dotenv import dotenv_values
from datetime import datetime
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import sqlite3
import requests

In [50]:
path_data=r'D:\AB_DB\AB_Cryptos_Files\X_To_AB'
path_x=r'D:\AB_DB\AB_Cryptos_Files\X_To_AB'

fund_path=r'D:\AB_DB\AB_Cryptos_Files\database\crypto_db.db'

path_config=r'D:\AB_DB\Script_ImportData'

today_now=datetime.now()
timeframe="4h" 

buildtime = datetime.now().strftime('%d%m%y_%H%M')

x_cryptocompare='y'

is_py=False

In [51]:
cols=["datetime","open","high","low","close","volume"]
cols_number=["open","high","low","close","volume"]
cols_last=["datetime","open","high","low","close","volume"]

cols_crypto_ohlc=["datetime","open","high","low","close",]
cols_crypto_vol=["datetime","volume"]

In [52]:
config_values= dotenv_values(dotenv_path=f'{path_config}\\.env')
api_key = config_values['bn_key']
api_secret =config_values['bn_secrete']
client=Client(api_key,api_secret)



In [53]:
cryptocompare_key=config_values['cryptocompare_key']
headers = {'Authorization': 'Bearer ' + cryptocompare_key}

In [54]:

ok=False

if is_py:
    print("Enter start date as yyyy-mm-dd(2019-01-01)")
    start_param = input("Enter start : ")

    print("Enter end date as yyyy-mm-dd(2022-01-01) if today ,press enter")
    end_param = input("Enter end : ") or ''
    
    print("Load Cryptocompare y|n")
    x_cryptocompare = input("Enter: ") or 'y'

    start=start_param
    if end_param == '':
     end=today_now.strftime("%Y-%m-%d")
    else:
     end=end_param  


    try:
        start_dt=datetime.strptime(start, "%Y-%m-%d")
        end_dt=datetime.strptime(end,"%Y-%m-%d")
        if start_dt>=end_dt:
           raise Exception(f"Error : {start_dt} can't be greater or equal to {end_dt}")
    except Exception as ex:
        raise ex



    print(f"Do you want to retrive data from {start_dt} to {end_dt} ?")
    press_y=input(f"Press y=True and n=False : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()
else: 
    start='2020-01-01'
    end= today_now.strftime("%Y-%m-%d")
    
limit_days=(( datetime.strptime(end,"%Y-%m-%d") )- ( datetime.strptime(start, "%Y-%m-%d") )).days 
limit_days=limit_days+1
    
print(f"{start} - {end}={limit_days}")

2020-01-01 - 2022-05-05=856


In [55]:
print("Connect SQLite")
conn = sqlite3.connect(os.path.abspath(fund_path))


sql="select * from crypto_meta_info where is_new = 1 and api_source='binance'"
#sql="select * from crypto_meta_info where  is_new=0 and is_active=1 and api_source='binance'"

sql_cpx="select * from crypto_meta_info where  is_new=1  and api_source='cryptocompare'"
#sql_cpx="select * from crypto_meta_info where  is_new=0 and is_active=1 and api_source='cryptocompare'"

sql_valid=False
try:
 print("Load Binance")
 
 dfCryptos_MetaData = pd.read_sql_query(sql, conn)
 dfCryptos_MetaData=dfCryptos_MetaData[['name','symbol','category','pair','api_source']]   
 print(dfCryptos_MetaData.info())
 print(dfCryptos_MetaData.head())   
    
    
 print("Load Cryptocompare")
 dfCompare_MetaData = pd.read_sql_query(sql_cpx, conn)  
 dfCompare_MetaData=dfCompare_MetaData[['name','symbol','category','pair','all_volume','api_source']]
 print(dfCompare_MetaData.info())
 print(dfCompare_MetaData.head())   


 sql_valid=True 

except Exception as ex:
 raise  ex




Connect SQLite
Load Binance
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        0 non-null      object
 1   symbol      0 non-null      object
 2   category    0 non-null      object
 3   pair        0 non-null      object
 4   api_source  0 non-null      object
dtypes: object(5)
memory usage: 0.0+ bytes
None
Empty DataFrame
Columns: [name, symbol, category, pair, api_source]
Index: []
Load Cryptocompare
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        4 non-null      object
 1   symbol      4 non-null      object
 2   category    4 non-null      object
 3   pair        4 non-null      object
 4   all_volume  4 non-null      int64 
 5   api_source  4 non-null      object
dtypes: int64(1), object(5)
memory usage

# Binance API

In [24]:
list_not_found=[]
# get data from Binance API
for index,item in  dfCryptos_MetaData.iterrows():
  try:  
    symbol=item['symbol']
    stablecoin=item['pair']
    name=item['name']
    product=f"{symbol}{stablecoin}"
    # check bn_symbols.txt
    print(f"{symbol} : {start} and {end}")
    df= pd.DataFrame(client.get_historical_klines(product,timeframe,start,end))
    
    df_ohlcv=df.iloc[:,:6]
    df_ohlcv.columns=cols

    df_ohlcv["datetime"]=pd.to_datetime(df_ohlcv["datetime"],unit="ms")
    df_ohlcv[cols_number]=df_ohlcv[cols_number].astype("float")
    df_ohlcv=df_ohlcv[cols_last]
    
    print(df_ohlcv.head())
    print(df_ohlcv.tail())
    
    file_data=f'{symbol}.csv'
    path_file=os.path.join(path_data,file_data)
    df_ohlcv.to_csv(path_file, header=True, index=False)
    #file_data=f'{symbol}.txt'
    #df_ohlcv.to_csv(path_file, header=True, index=False, sep=',', mode='a')
   

    
  except Exception as ex:
    print(f'{symbol}: {ex}')
    list_not_found.append(f'{symbol} - {name}')
    
print('list not found symbol')
for item in list_not_found:
 print(item)

SUSHI : 2020-01-01 and 2022-05-05
             datetime    open    high    low   close       volume
0 2020-09-01 08:00:00   6.080  15.970  6.080  10.680  1973917.073
1 2020-09-01 12:00:00  10.679  11.490  9.350  10.220  3052057.999
2 2020-09-01 16:00:00  10.220  10.917  9.805  10.166  1322667.328
3 2020-09-01 20:00:00  10.168  10.769  8.600   8.804  1823641.839
4 2020-09-02 00:00:00   8.806   9.500  6.749   8.039  2769022.552
                datetime   open   high    low  close     volume
3660 2022-05-04 08:00:00  2.410  2.470  2.410  2.427  1385013.3
3661 2022-05-04 12:00:00  2.427  2.445  2.361  2.382  1055686.2
3662 2022-05-04 16:00:00  2.381  2.589  2.372  2.575  2536269.3
3663 2022-05-04 20:00:00  2.576  2.651  2.552  2.618   999521.4
3664 2022-05-05 00:00:00  2.618  2.636  2.580  2.600   809979.2
list not found symbol


# Cryptocompare API

In [56]:
if x_cryptocompare=='y':

    x_not_found=[]
    for index,item in  dfCompare_MetaData.iterrows():

     try:  
        name=item['name']
        # name='BTC'

        symbol=item['symbol']

        pair=item['pair']
        required_all_vols=item['all_volume']

        url = f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={name}&tsym={pair}&limit={limit_days}"
        print(url)
        json_ohlc=requests.get(url, headers=headers).json()

        df_ohlc=pd.DataFrame(json_ohlc['Data']['Data'])
        df_ohlc=df_ohlc.query('close>0')
        df_ohlc['datetime'] = [datetime.fromtimestamp(d) for d in df_ohlc.time]
        if required_all_vols==1:

            df_ohlc=df_ohlc[cols_crypto_ohlc]
            print("Require all volume")
            url_vol=f"https://min-api.cryptocompare.com/data/symbol/histoday?fsym={name}&tsym={pair}&limit={limit_days}"
            print(url_vol)
            json_vol=requests.get(url_vol, headers=headers).json()   

            df_vol=pd.DataFrame(json_vol['Data'])
            df_vol['datetime'] = [datetime.fromtimestamp(d) for d in df_vol.time]
            df_vol=df_vol.drop(columns=['time'])
            df_vol=df_vol[['datetime','total_volume_total']]
            df_vol=df_vol.rename(columns={'total_volume_total':'volume'})

            print(df_vol.info())
            print(df_vol.head())

            df_cypto_compare=pd.concat([df_ohlc,df_vol['volume']],axis=1)

        elif required_all_vols==0:

          df_ohlc=df_ohlc.rename(columns={'volumeto':'volume'})
          df_cypto_compare=df_ohlc[cols_crypto_ohlc+['volume']]

        print(df_cypto_compare.info())
        print(df_cypto_compare.head())


        xfile_data=f'{symbol}.csv'
        xpath_file=os.path.join(path_x,xfile_data)
        df_cypto_compare.to_csv(xpath_file, header=True, index=False)

     except Exception as ex:
          print(f'{symbol}: {ex}')
          x_not_found.append(f'{symbol} - {name}')



https://min-api.cryptocompare.com/data/v2/histoday?fsym=APE&tsym=USD&limit=856
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 807 to 856
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  50 non-null     datetime64[ns]
 1   open      50 non-null     float64       
 2   high      50 non-null     float64       
 3   low       50 non-null     float64       
 4   close     50 non-null     float64       
 5   volume    50 non-null     float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 2.7 KB
None
               datetime   open   high     low  close       volume
807 2022-03-17 07:00:00  10.60  10.60  10.600  10.60   3245178.31
808 2022-03-18 07:00:00  10.60  10.60  10.080  10.08  38533693.34
809 2022-03-19 07:00:00  10.08  10.08  10.080  10.08         0.00
810 2022-03-20 07:00:00  10.08  10.08  10.080  10.08         0.00
811 2022-03-21 07:00:00  10.08  10.82   9.749  10.53  